In [1]:
import numpy as np
from sklearn.model_selection import KFold

In [2]:
x = np.array([
    [1,2],[3,4],[1,2],[3,4]
])

y = np.array([1,2,3,4])


In [3]:
x

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

In [4]:
y

array([1, 2, 3, 4])

In [12]:
kf = KFold(n_splits = 2)
print(kf.get_n_splits(x))
print(kf)


2
KFold(n_splits=2, random_state=None, shuffle=False)
<generator object _BaseKFold.split at 0x000001DBDCCD2740>


In [17]:
for train_idx, test_idx in kf.split(x):
    print(f'train:{train_idx}')
    print(f'test:{test_idx}')
    

train:[2 3]
test:[0 1]
train:[0 1]
test:[2 3]


In [18]:
for train_idx, test_idx in kf.split(x):
    print(f'train:{train_idx}')
    print(f'test:{test_idx}')
    print('----train_data')
    print(x[train_idx])
    print('---validation_data') 
    print(x[test_idx])  

train:[2 3]
test:[0 1]
----train_data
[[1 2]
 [3 4]]
---validation_data
[[1 2]
 [3 4]]
train:[0 1]
test:[2 3]
----train_data
[[1 2]
 [3 4]]
---validation_data
[[1 2]
 [3 4]]


In [20]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [32]:
red = pd.read_csv('C:/Users/bigdata/Documents/ds_study/data/practice/winequality-red.csv',sep = ';')
white = pd.read_csv('C:/Users/bigdata/Documents/ds_study/data/practice/winequality-white.csv',sep = ';')
red['color'] =1
white['color'] = 0

wine = pd.concat([red, white])

wine['taste'] = [1. if grade > 5 else 0 for grade in wine['quality']]

x = wine.drop(['taste','quality'],axis = 1)
y = wine['taste']

x_tr, x_t, y_tr, y_t = train_test_split(x,y,test_size = 0.2, random_state = 13)

wine_tree = DecisionTreeClassifier(max_depth = 2, random_state = 0xC0FFEE)
wine_tree.fit(x_tr, y_tr)

y_pred_tr = wine_tree.predict(x_tr)
y_pred = wine_tree.predict(x_t)

acc_tr = accuracy_score(y_tr, y_pred_tr)
acc_t = accuracy_score(y_t, y_pred)

print(f'Train acc : {acc_tr}')
print(f'Test acc : {acc_t}')

Train acc : 0.7294593034442948
Test acc : 0.7161538461538461


In [33]:
len(wine)

6497

In [34]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5)
wine_tree_cv = DecisionTreeClassifier(max_depth = 2, random_state = 13)

In [35]:
for train_idx, val_idx in kf.split(x):
    print(len(train_idx), len(val_idx))

5197 1300
5197 1300
5198 1299
5198 1299
5198 1299


In [37]:
for train_idx, test_idx_idx in kf.split(x):
    x_tr = x.iloc[train_idx]
    x_t = x.iloc[test_idx]
    y_tr = y.iloc[train_idx]
    y_t = y.iloc[test_idx]

    wine_tree_cv.fit(x_tr,y_tr)
    y_pred = wine_tree_cv.predict(x_t)
    print(f'acc: {accuracy_score(y_t, y_pred)}')

acc: 0.5
acc: 0.5
acc: 0.5
acc: 0.5
acc: 0.5


In [40]:
from sklearn.model_selection import StratifiedKFold

kf = KFold(n_splits = 5)
wine_tree_cv = DecisionTreeClassifier(max_depth = 2, random_state = 13)

cv_acc = []

for train_idx, test_idx_idx in kf.split(x):
    x_tr = x.iloc[train_idx]
    x_t = x.iloc[test_idx]
    y_tr = y.iloc[train_idx]
    y_t = y.iloc[test_idx]

    wine_tree_cv.fit(x_tr,y_tr)
    y_pred = wine_tree_cv.predict(x_t)

    cv_acc.append(accuracy_score(y_t, y_pred))

cv_acc

[0.5, 0.5, 0.5, 0.5, 0.5]

In [39]:
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits = 5)
wine_tree_cv = DecisionTreeClassifier(max_depth = 2, random_state = 13)

cross_val_score(wine_tree_cv, x, y, cv = kf)

array([0.60076923, 0.68846154, 0.70900693, 0.76289453, 0.78675905])

In [41]:
def skfolder(depth):
    from sklearn.model_selection import cross_val_score

    kf = KFold(n_splits = 5)
    wine_tree_cv = DecisionTreeClassifier(max_depth = depth, random_state = 13)

    print(cross_val_score(wine_tree_cv, x, y, cv = kf))

In [42]:
skfolder(5)

[0.67923077 0.66692308 0.7113164  0.77213241 0.79214781]


In [45]:
from sklearn.model_selection import cross_validate

cross_validate(wine_tree_cv, x, y, cv =kf, return_train_score= True)

{'fit_time': array([0.00812197, 0.00799918, 0.00851417, 0.00550652, 0.00650692]),
 'score_time': array([0.00150418, 0.00124145, 0.00200415, 0.00199914, 0.00099969]),
 'test_score': array([0.60076923, 0.68846154, 0.70900693, 0.76289453, 0.78675905]),
 'train_score': array([0.74427554, 0.74831634, 0.74451712, 0.73143517, 0.72508657])}

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_depth': [2,4,7,10]}
wine_tree = DecisionTreeClassifier(max_depth = 2, random_state = 13)

gridsearch = GridSearchCV(estimator = wine_tree, param_grid = params, cv = 5)
gridsearch.fit(x,y)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=2, random_state=13),
             param_grid={'max_depth': [2, 4, 7, 10]})

In [49]:
import pprint 

pp = pprint.PrettyPrinter(indent =4)
pp.pprint(gridsearch.cv_results_)

{   'mean_fit_time': array([0.00683308, 0.0119668 , 0.02209239, 0.03390045]),
    'mean_score_time': array([0.00190406, 0.0017004 , 0.00150719, 0.00190387]),
    'mean_test_score': array([0.6888005 , 0.66356523, 0.65340854, 0.64401587]),
    'param_max_depth': masked_array(data=[2, 4, 7, 10],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object),
    'params': [   {'max_depth': 2},
                  {'max_depth': 4},
                  {'max_depth': 7},
                  {'max_depth': 10}],
    'rank_test_score': array([1, 2, 3, 4]),
    'split0_test_score': array([0.55230769, 0.51230769, 0.50846154, 0.51615385]),
    'split1_test_score': array([0.68846154, 0.63153846, 0.60307692, 0.60076923]),
    'split2_test_score': array([0.71439569, 0.72363356, 0.68360277, 0.66743649]),
    'split3_test_score': array([0.73210162, 0.73210162, 0.73672055, 0.71054657]),
    'split4_test_score': array([0.75673595, 0.7182448 , 0.73518091, 0.72517321]),
    'std

In [50]:
gridsearch.best_estimator_

DecisionTreeClassifier(max_depth=2, random_state=13)

In [52]:
gridsearch.best_score_

0.6888004974240539

In [53]:
gridsearch.best_params_

{'max_depth': 2}

In [54]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

estimators = [
    ('scaler',StandardScaler()),
    ('clf', DecisionTreeClassifier())
]
pipe = Pipeline(estimators)

In [56]:
param_grid = [{'clf__max_depth': [2,4,6,8]}]

GridSearch = GridSearchCV(estimator = pipe , param_grid = param_grid, cv =5)
GridSearch.fit(x,y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', DecisionTreeClassifier())]),
             param_grid=[{'clf__max_depth': [2, 4, 6, 8]}])

In [57]:
gridsearch.best_estimator_

DecisionTreeClassifier(max_depth=2, random_state=13)

In [60]:
import pandas as pd

score_df = pd.DataFrame(GridSearch.cv_results_)
score_df[['params','rank_test_score','mean_test_score','std_test_score']]

,params,rank_test_score,mean_test_score,std_test_score
0,{'clf__max_depth': 2},1,0.688800,0.071799
1,{'clf__max_depth': 4},2,0.663565,0.083905
2,{'clf__max_depth': 6},3,0.660333,0.082192
3,{'clf__max_depth': 8},4,0.655253,0.080230
